In [ ]:
import os
import openai
import sys
import json
import re
from IPython.display import display, HTML
sys.path.append('../..')

In [ ]:
openai.api_key = os.environ['OPENAI_API_KEY']

## LangChain Imports

In [ ]:
from langchain_core.output_parsers import StrOutputParser

# https://python.langchain.com/docs/modules/memory/adding_memory
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain.schema import SystemMessage
from langchain_openai import ChatOpenAI

## Setup and Initialize Chain

In [ ]:
system_v1_template = """You are an intelligent Cognitive UI agent that is having a conversation with an user.
Your task is to have a conversation and respond appropriately to the user. You have the ability to generate UI components 
in the form of HTML code and HTML is the output format that the user prefers when needed.
Before you output any HTML, you need to confirm that the HTML is valid, stylized with CSS, JS, and/or other frameworks and 
most importantly, it must satisfy the user's requirements. 
Valid HTML must follow the standard HTML file format containing the keyword 'DOCTYPE'.

If the user provides feedback to the generated HTML, you will incorporate their feedback
and make new attempts until they're satisfied. In order to generate HTML, you may need content from the user and it is okay to 
collect information from the user based on the type of UI component they need. 

Once you have enough information from the user, you need to output HTML code; you should
also do this if the user is demanding for HTML. However, if you need more information, you should continue asking for information.
If you have already generated HTML and the user provides feedback and suggest changes, please respond with the updated HTML.
If you say you will be generating HTML, you must do so.
"""

In [ ]:
def make_chat(system_template, verbose_mode=False):
    prompt = ChatPromptTemplate.from_messages(
        [
            SystemMessage(
                content=system_template
            ),  # The persistent system prompt
            MessagesPlaceholder(
                variable_name="chat_history"
            ),  # Where the memory will be stored.
            HumanMessagePromptTemplate.from_template(
                "{human_input}"
            ),  # Where the human input will injected
        ]
    )
    
    llm = ChatOpenAI(temperature = 0)
    mem = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
    mem.clear()
    
    chat_llm_chain = LLMChain(
        llm=llm,
        prompt=prompt,
        verbose=verbose_mode,
        memory=mem,
    )
    return chat_llm_chain

In [ ]:
html_pattern = re.compile(r'<!DOCTYPE html>.*?</html>', re.DOTALL)
def interact(model, input):
    o = model.predict(human_input=input)
    html_match = html_pattern.search(o)
    s = False
    if html_match:
        s = True
        display(HTML(html_match.group(0)))
    if s:
        o = re.sub(r"```html.*?```", "", o, flags=re.DOTALL)
    return o

### Testing

In [ ]:
model = make_chat(system_v1_template, verbose_mode=False)
interact(model, "Hi")

In [ ]:
interact(model, "I want to make a form on my website")

In [ ]:
interact(model, """I need text fields for their first name, last name, and email address. 
I also need a textbox for the user to input a custom message.""")

In [ ]:
interact(model, """
The labels should look fancy and each have different colors that look great together. The label's text should indicate the content 
that the user should put into the text field. The labels must have a Bootstrap Glyphicon representing that particular field.
""")

In [ ]:
interact(model, """I want all the text fields to be on the same row. The textbox on its own row. 
I also want a submit button centered in its row for the form. Now, show me the form""")

In [ ]:
interact(model, "Make the form 500 pixels wide and give it a fancy gradient background color. The form should have rounded borders")

In [ ]:
interact(model, "Make the submit button take up the entire space on its row and make it also rounded corners")

In [ ]:
interact(model, "Looks fine for now, ignore the 500px width rule and add 30px padding around the form")

In [ ]:
interact(model, "The text labels must have a Bootstrap Glyphicon representing that particular field")

In [ ]:
interact(model, "the form does not have Glyphicons next to the text field labels, please add the correct icons")

In [ ]:
interact(model, """your Glyphicons do not work. Use bootstrap emoji-like Glyphicons for names and email. 
A person icon for the names and an email icon for the email address""")

In [ ]:
interact(model, "alright, now make the form responsive and allow it to be resized when dragging the corner of the form")

In [ ]:
interact(model, """here is the link to an image of mount everest

https://upload.wikimedia.org/wikipedia/commons/1/1a/Shivas_Kinder_-_0253.jpg

insert the image with the above url as the source between the custom message and submit button on my form
""")

In [ ]:
interact(model, "thanks for the form, I am done with the form, but now I need a main page screen with the same mount everest image as the background and white text centered vertically and horizontally on the page. The text should be 'Mount'")

In [ ]:
interact(model, "make me a content container that is split into 4 boxes with colors red, blue, green, and yellow")

In [ ]:
interact(model, "okay bye")